### Import Section

In [1]:
import brightway2 as bw
bw.projects.set_current("pulpo")

To import the methanol inventories from the [One Earth](https://doi.org/10.1016/j.oneear.2021.04.001) paper you will need to have a the ecoinvent cutoff 3.8 database in your project, named "cutoff38".

In [2]:
if "one_earth_38" not in bw.databases:
    bw.BW2Package.import_file("data/one_earth_38.bw2package")
    print("Imported the One Earth inventories!")
else:
    print("One Earth inventories already loaded.")

One Earth inventories already loaded.


In [3]:
import os
import sys
sys.path.append('../')
from pulpo import pulpo

import pandas as pd
pd.set_option('display.max_colwidth', None)

You may want to use the premise adjusted IPCC method:

In [4]:
#from premise_gwp import add_premise_gwp
#add_premise_gwp()  

<div style="text-align: center; background-color: #f0f0f0; padding: 10px;">
    <h2 style="font-family: 'Arial', sans-serif; font-weight: bold; color: #555;">(1) Selection of LCI Data</h2>
</div>

### Setup

In [5]:
project = "pulpo"
database = "one_earth_38"
methods = "('IPCC 2013', 'climate change', 'GWP 100a')" #, incl. H')"

# Substitute with your working directory of choice
notebook_dir = os.path.dirname(os.getcwd())
directory = os.path.join(notebook_dir, 'data')

# Substitute with your GAMS path
GAMS_PATH = "C:/GAMS/37/gams.exe"

In [6]:
pulpo_worker = pulpo.PulpoOptimizer(project, database, methods, directory)

In [7]:
pulpo_worker.get_lci_data()

<div style="text-align: center; background-color: #f0f0f0; padding: 10px;">
    <h2 style="font-family: 'Arial', sans-serif; font-weight: bold; color: #555;">(2) User Specifications</h2>
</div>

<div style="text-align: center;">
    <img src="pictures/one_earth_showcase.png" alt="Image Alt Text" style="width: 80%;" />
</div>

### Specify the **functional unit**

In [8]:
methanol = pulpo_worker.retrieve_activities(activities=["market for methanol"])

In [9]:
demand = {methanol[0]: 100}

### Specify the **choices**

In [10]:
electricity = pulpo_worker.retrieve_activities(reference_products=["electricity"])

Exclude BECCS for now:

In [11]:
#electricity = [elec for elec in electricity if elec['name'] != 'BECCS']

In [12]:
CO2 = pulpo_worker.retrieve_activities(reference_products=["CO2"])

In [13]:
H2 = pulpo_worker.retrieve_activities(reference_products=["H2"])

In [14]:
methanol = pulpo_worker.retrieve_activities(reference_products=["methanol"])

In [15]:
choices  = {'electricity': {elec: 1e6 for elec in electricity},
            'CO2': {co2: 1e6 for co2 in CO2},
            'H2': {h2: 1e6 for h2 in H2},
            'methanol': {meoh: 1e6 for meoh in methanol},}

<div style="text-align: center; background-color: #f0f0f0; padding: 10px;">
    <h2 style="font-family: 'Arial', sans-serif; font-weight: bold; color: #555;">(3) Solution</h2>
</div>

In [16]:
instance = pulpo_worker.instantiate(choices=choices, demand=demand)

Creating Instance
Instance created


In [ ]:
results = pulpo_worker.solve(GAMS_PATH=GAMS_PATH)

GAMS solvers library availability: True
Solver path: C:\GAMS\37\gams.exe


In [ ]:
pulpo_worker.summarize_results(choices=choices, demand=demand, zeroes=True)

In [ ]:
pulpo_worker.save_results(choices=choices, demand=demand, name='one_earth_results.xlsx')